<a href="https://colab.research.google.com/github/bksat90/StockPrediction/blob/main/StockPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.layers import LSTM, SimpleRNN, Dense
from keras.utils import plot_model
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [10]:
# Fetch stock data from Yahoo Finance
ticker = 'IBM'
# 2 years data
data = yf.download(ticker, start="2021-12-01", end="2023-11-30")

# printing the head of the imported data
data.head()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-12-01,118.250000,118.930000,116.849998,116.919998,106.124680,5958300
2021-12-02,117.370003,117.980003,116.559998,116.900002,106.106529,5267100
2021-12-03,117.360001,119.360001,117.360001,118.839996,107.867416,6630100
2021-12-06,119.400002,121.150002,119.400002,119.910004,108.838631,4785600
2021-12-07,120.480003,122.080002,120.070000,121.580002,110.354416,5194000


In [12]:
# seperate the close data which will be used for prediction
close_data = data['Close'].values.reshape(-1,1)

In [15]:
# preprocessing - rescaling the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(close_data)

## Creating the time series data structure with structure N * T * D
# N = number of samples, T = block/series size = look_back, D = features = 1
look_back = 40

# 75% data is considered as train data
train_size = int(len(scaled_data) * 0.75)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size - look_back:]

# generates time series data
train_generator = TimeseriesGenerator(train_data, train_data,
                                     length=look_back, batch_size=20)
test_generator = TimeseriesGenerator(test_data, test_data,
                                     length=look_back, batch_size=1)